In [1]:
import polars as pl
import json
from web3 import Web3
from eth_abi import decode as decode_abi

In [2]:
with open("v3_abi.json", "r") as f:
    abi = json.load(f)
abi = json.loads(abi['result'])


web3 = Web3(Web3.HTTPProvider("https://eth.llamarpc.com"))

In [3]:
df = pl.read_parquet("../../sample.parquet")

print(df.head())
print(df.columns)

shape: (5, 10)
┌────────────┬────────────┬───────────┬───────────┬───┬───────────┬───────────┬────────┬───────────┐
│ block_numb ┆ transactio ┆ log_index ┆ transacti ┆ … ┆ topic1    ┆ topic2    ┆ topic3 ┆ data      │
│ er         ┆ n_index    ┆ ---       ┆ on_hash   ┆   ┆ ---       ┆ ---       ┆ ---    ┆ ---       │
│ ---        ┆ ---        ┆ u32       ┆ ---       ┆   ┆ binary    ┆ binary    ┆ binary ┆ binary    │
│ u32        ┆ u32        ┆           ┆ binary    ┆   ┆           ┆           ┆        ┆           │
╞════════════╪════════════╪═══════════╪═══════════╪═══╪═══════════╪═══════════╪════════╪═══════════╡
│ 17780001   ┆ 117        ┆ 411       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]     ┆        ┆ data]     │
│ 17780006   ┆ 109        ┆ 311       ┆ [binary   ┆ … ┆ [binary   ┆ [binary   ┆ null   ┆ [binary   │
│            ┆            ┆           ┆ data]     ┆   ┆ data]     ┆ data]   

In [4]:
Web3.to_hex(df[0, "data"])

'0x00000000000000000000000000000000000000000000000000000000b2cdc211ffffffffffffffffffffffffffffffffffffffffffffffffe9d2ac163f58a0cb0000000000000000000000000000000000005a2e2a1fad4d10d613c658ef7a31000000000000000000000000000000000000000000000001d5669d4b0b0d4d2b00000000000000000000000000000000000000000000000000000000000310f5'

In [5]:
!cast sig-event "Mint(address sender, address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Burn(address indexed owner, int24 indexed tickLower, int24 indexed tickUpper, uint128 amount, uint256 amount0, uint256 amount1)"
!cast sig-event "Swap(address indexed sender, address indexed recipient, int256 amount0, int256 amount1, uint160 sqrtPriceX96, uint128 liquidity, int24 tick)"

0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde
0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c
0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67


In [6]:
mint_event_bytes = Web3.to_bytes(0x7a53080ba414158be7ec69b987b5fb7d07dee101fe85488f0853ae16239d0bde)
burn_event_bytes = Web3.to_bytes(0x0c396cd989a39f4459b5fa1aed6a9a8dcdbc45908acfd67e028cd568da98982c)
swap_event_bytes = Web3.to_bytes(0xc42079f94a6350d7e6235f29174924f928cc2ac818eb64fed8004e115fbcca67)

event_df = df.filter((pl.col("topic0") == mint_event_bytes) | (pl.col("topic0") == burn_event_bytes) | (pl.col("topic0") == swap_event_bytes))
print("All events: ", event_df.shape)
print("Mint events: ", event_df.filter(pl.col("topic0") == mint_event_bytes).shape)
print("Burn events: ", event_df.filter(pl.col("topic0") == burn_event_bytes).shape)
print("Swap events: ", event_df.filter(pl.col("topic0") == swap_event_bytes).shape)
event_df.head(8)

All events:  (246, 10)
Mint events:  (4, 10)
Burn events:  (4, 10)
Swap events:  (238, 10)


block_number,transaction_index,log_index,transaction_hash,contract_address,topic0,topic1,topic2,topic3,data
u32,u32,u32,binary,binary,binary,binary,binary,binary,binary
17780001,117,411,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780006,109,311,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780008,13,83,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780009,71,171,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780018,65,203,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,1,14,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780021,2,21,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]
17780022,157,361,[binary data],[binary data],[binary data],[binary data],[binary data],null,[binary data]


In [7]:
mint_decode_types = ["address", "uint128", "uint256", "uint256"]
burn_decode_types = ["uint128", "uint256", "uint256"]
swap_decode_types = ["int256", "int256", "uint160", "uint128", "int24"]

In [8]:
# loop over the events and simulate the actions against the pools
for row in event_df.iter_rows(named=True):
    if row['topic0'] == mint_event_bytes:
        (sender, amount, amount0, amount1) = decode_abi(mint_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
    elif row['topic0'] == burn_event_bytes:
        (amount, amount0, amount1) = decode_abi(burn_decode_types, row['data'])
        owner = decode_abi(["address"], row['topic1'])
        tickLower = decode_abi(["int24"], row['topic2'])
        tickUpper = decode_abi(["int24"], row['topic3'])
    elif row['topic0'] == swap_event_bytes:
        (amount0, amount1, sqrtPriceX96, liquidity, tick) = decode_abi(swap_decode_types, row['data'])
        sender = decode_abi(["address"], row['topic1'])
        recipient = decode_abi(["address"], row['topic2'])